Analysis of eye tracking files from Spiro study.
Gaze dispersion metric based on Christoforou et al, 2015: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4428128/pdf/fpsyg-06-00579.pdf
The objective is as follows:
(1) compute within-subject dispersion metric, based on short sections of the film (250 ms with 50 ms shift, ie 80% overlap)
(2) get sections with extreme dispersion score


In [1]:
# analyse film data, v3 Oct/Nov 2020

# read and prep Tobii .tsv files from CortEx study
import os
from datetime import datetime
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import nanmedian, nanmean, nanstd
from scipy.spatial import distance
from scipy import stats
from sklearn.metrics import pairwise_distances
import warnings
warnings.filterwarnings('ignore')

Utils

In [2]:
from scipy.stats import norm
# get participant numbers

def get_pnums(infiles):
    """
    Get participant numbers from filenames.

    Parameters
    ----------
    infiles: array of input file names

    Returns
    -------
    list of participant numbers 
    """
    id = []
    for i,filenames in enumerate(infiles):
        pnum = [int(s) for s in filenames.split('_') if s.isdigit()]
        id.append([pnum])

def flatten(list_name):
    """ 
    Function that flattens list of lists.
    Source: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists

    Parameters
    ----------
    list_name: str
        Name of the list to flatten
    
    Returns
    -------
    Flattened list
    """
    flattened = [item for sublist in list_name for item in sublist]
    return flattened



def get_random_samps(df, frame_num_max, id):
    """ 
    Fit a participant's gaze data to normal distribution.
    Randomly sample normal distribution with participant-specific mean and std.

    Parameters
    ----------
    df: pandas dataframe
        dataframe holding eye tracking data from all participants

    frame_num_max: int
        maximum number of frames for participant, will be used to determine how many samples to draw

    id: int
        participant id 
    
    Returns
    -------
    Simulated gaze data in x and y.

    """
    # first, let's simulate some gaze data.
    # some basic parameters
    #gaze_length = df.frame_num.max() # number of samples we need to generate
    mux,stdx = norm.fit(df.loc[df.id == id,'gaze_in_x'])
    muy,stdy = norm.fit(df.loc[df.id == id, 'gaze_in_y'])
    # get samples
    sim_in_x = norm.rvs(loc = mux, scale = stdx, size = frame_num_max)
    sim_in_y = norm.rvs(loc = muy, scale = stdy, size = frame_num_max)
    return sim_in_x, sim_in_y




In [3]:
class Tobii_file:
    def __init__(self, filename, film_dur_s, screenres_x = None, screenres_y = None, s_rate= None):
        """
    
        """
        if not screenres_x:
            self.screen_resx = 1280
        if not screenres_y:
            self.screen_resy = 1024
        if not s_rate:
            s_rate = 60
        self.filename = filename
        self.screen_res = [screenres_x, screenres_y]
        self.s_rate = s_rate
        self.film_dur_s = film_dur_s
        self.pnum = [int(s) for s in filename.split('_') if s.isdigit()]

    def get_calibration(self, calibration_filename, calibration_dir):
        """
        Get calibration details (accuracy, precision).

        Parameters
        ----------
        filename: str
            Calibration filename
        calibration file directory: str
            path to calibration file directory
        
        Returns
        -------
        

        """
        self.calibration_filename = calibration_filename
        calibration_df = pd.read_csv(os.path.join(calibration_dir,calibration_filename))
        if calibration_df.loc[calibration_df['used']=='used','used'].empty:
            self.calibration = 'unused'
        else:
            self.calibration = calibration_df.loc[calibration_df['used'] == 'used',:]



(i) get time stamps from message file
- discard everything but the messages containing frame nr.
(ii) load data file and add column for frame nr
(iii) label rows between message (framenr) time stamps with the appropriate frame nr

In [8]:
home = 1 
if home:
    rawfilepath = r"C:\Users\Luzia T\Eye-gaze-divergence"
    Tobii_files = [f for f in os.listdir(rawfilepath) if 'merged' in f]
    calibration_files = [f for f in os.listdir(rawfilepath) if 'calib' in f]
else:
    # first we need to establish where to find the files and read them in.
    rawfilepath = r"P:\Spironolactone\eye_tracking\Tobii"
    # get merged Tobii files - these have both event info and eye gaze data.
    Tobii_files = [f for f in os.listdir(rawfilepath) if 'merged' in f]
    # get_calibration files
    calibration_files = [f for f in os.listdir(rawfilepath) if 'calib' in f and f.endswith('.tsv')]
    msg_files = [f for f in os.listdir(rawfilepath) if 'msg' in f and f.endswith('.tsv')]

In [8]:
# get pnum, left/right gaze point on display area, event messages 
# keep only columns between framenum 0 and the last frame
# set invalid samples to NaN (validity == 0)
frames = []
gaze_in_x = []
gaze_in_y = []
pnums = []

for i,filename in enumerate(Tobii_files):
    # read file
    tobii_file = pd.read_table(os.path.join(rawfilepath, filename))
    # get participant number and add as column
    tobii_file['pnum'] = np.repeat([int(s) for s in filename.split('_') if s.isdigit()], tobii_file.shape[0])
    # get row index of frame each frame presentation
    event_index = tobii_file.loc[tobii_file.msg.str.contains('FRAME',na = False),'msg'].index
    # drop everything before first frame index (ie frame 0) and after last frame
    tobii_file = tobii_file.loc[event_index[0]:event_index[-1],:]
    # extract frame number
    tobii_file.loc[event_index, 'msg'] = [f[0] for f in tobii_file.loc[event_index,'msg'].str.split(';', n = 1)]
    tobii_file['frame_num'] = tobii_file.loc[event_index,'msg'].apply(lambda x: re.findall(r'\d',x)).apply(''.join).astype('int')
    # fill the rows between event markers with the appropriate frame number
    tobii_file.loc[:,'frame_num'] = tobii_file.loc[:,'frame_num'].fillna(method = 'ffill')
    # drop event markers
    tobii_file = tobii_file.drop(labels = event_index,axis = 0)
    # discard invalid samples
    tobii_file = tobii_file.loc[(tobii_file.right_gaze_point_validity == 1)&(tobii_file.left_gaze_point_validity == 1),:]
    # group by frame number and calculate mean
    tobii_file = tobii_file.groupby('frame_num').mean()
    tobii_file = tobii_file.reset_index()
    # collect data into lists
    pnums.append(tobii_file.pnum.values)
    gaze_in_x.append(tobii_file.right_gaze_point_on_display_area_x.values)
    gaze_in_y.append(tobii_file.left_gaze_point_on_display_area_y.values)
    frames.append(tobii_file.frame_num.values)
    

We create one big dataframe from the above.
This is already ~ half a million rows for only 24 participants (with max 1 row per frame). This is going to become unwieldy for a significantly larger number of participants...may need to think about restructuring/parallel computing/cloud use

In [9]:
# create dataframe
et_df = pd.DataFrame({'id':flatten(pnums),'gaze_in_x':flatten(gaze_in_x),'gaze_in_y':flatten(gaze_in_y),'frame_num':flatten(frames)})

We now have valid data for left/right gaze positions (on display area) for all participants in a single data frame. We now want to get a measure of divergence. We need to bear in mind a couple of things.
First of all, calibration accuracy/precision will be different between participants. The first thing to check is to make sure that the precision is roughly the same. If not, this could really affect the divergence measure in a fairly unpredictable way, so we would need to exclude those subjects. Next, we want to have a look at accuracy. This is less of an issue because it is basically just an offset. We could try to correct gaze point data for each participant, but this will not be needed as divergent frames are identified based on comparing to a 'random' sequence of gaze data (which we will simulate below). Unless the offset is extreme, this should therefore not affect the results.
Because different participants are missing different frames and because we don't expect huge variation between individual frames, given a frame rate of ~30 fps, we will look at 250 ms windows, and we will use a sliding window approach, shifting the window 50 ms each time (resulting in 80% overlap between windows).
For each window and participant, we will calculate the Euclidean distance between the participant in question and every other participant/the participant in question and the random sequence. For any given window, we will then check whether this distance is greater in the former case than in the latter. If so, the window is identified as 'divergent' for that participant.

First, let's simulate x/y data by sampling from a normal distribution, based on each participant's gaze data.

In [10]:
%%timeit
# create columns to hold simulated values
et_df['gaze_in_x_sim'] = np.nan
et_df['gaze_in_y_sim'] = np.nan

# fill with simulated gaze data in x and y for each participant
for i,pnum in enumerate(et_df.id.unique()):
    pnum_df = et_df.loc[et_df.id == pnum,['id','gaze_in_x','gaze_in_y']]
    max_frame = pnum_df.shape[0]
    simx,simy = get_random_samps(pnum_df,max_frame,pnum)
    et_df.loc[et_df.id == pnum, 'gaze_in_x_sim'] = simx
    et_df.loc[et_df.id == pnum, 'gaze_in_y_sim'] = simy

57.6 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
# first, let's construct a dataframe for the x and y gaze positions that holds data from all participants.
gaze_x = et_df.pivot(index = 'frame_num',columns = 'id', values = 'gaze_in_x')
gaze_y = et_df.pivot(index = 'frame_num',columns = 'id', values = 'gaze_in_y')
gaze_x_sim = et_df.pivot(index = 'frame_num',columns = 'id', values = 'gaze_in_x_sim')
gaze_y_sim = et_df.pivot(index = 'frame_num',columns = 'id', values = 'gaze_in_y_sim')

#et_df.loc[:,['frame_num','gaze_in_x','gaze_in_y']].groupby('frame_num').rolling(window = 15, min_periods = 3).apply(distance.euclidean)
dists = []
for i in gaze_y.index:
    frame_df = pd.concat([gaze_x.loc[i,:],gaze_y.loc[i,:]],axis = 1)
    frame_dist_val =  np.unique(pairwise_distances(frame_df.dropna(), frame_df.dropna(),force_all_finite = True)).mean()
    dists.append(frame_dist_val)

# now we calculate the mean euclidean distance for each window.
pwise_dists = pd.Series(dists, index = gaze_x.index)
windows = pwise_dists.rolling(window = 15)
steps = np.arange(0,len(dists)-15,step = 3)

# now calculate the mean for each 15-frame window (corresponds to ~250 ms), with a step size of 3 (80% overlap between successive windows)
wind_dists = []
for i in steps:
    wind_dist = np.array(dists)[i:i+15].mean()
    wind_dists.append(wind_dist)

In [22]:
# Now we calculate the distance between each individual's gaze location for each frame and the 'random' gaze positions we generated above by sampling from a normal distribution.

# calculcate Euclidean distance
dist_sim_df = np.sqrt(np.square(gaze_x-gaze_x_sim)+np.square(gaze_y-gaze_y_sim)).mean(axis=1)

# now calculate mean for each window. Can use the steps from above as the arrays are the same length.
wind_dists_sim = []
for i in steps:
    wind_dist_sim = dist_sim_df.dropna().iloc[i:i+15].mean()
    wind_dists_sim.append(wind_dist_sim)   

In [134]:
diff_act_sim = pd.Series(wind_dists)-pd.Series(wind_dists_sim)
# where the difference between actual and simulated dispersion scores is >0, window is divergent
divergent_winds = diff_act_sim[diff_act_sim>0]

In [101]:
# %age of divergent windows:
perc_div_winds = (len(divergent_winds)/len(diff_act_sim))*100
print("The percentage of divergent windows is:", perc_div_winds)

inds_div_winds = divergent_winds.index

The percentage of divergent windows is: 28.065494238932686


In [136]:
# get consecutive sections of divergent windows.This is so we can relate it to sections of the film.
divergent_winds = divergent_winds.reset_index()
divergent_winds = divergent_winds.rename(columns = {'index':'div_winds'})
divwinds_shifted = divergent_winds.shift(-1).reset_index(drop = True)
divwinds_shifted = divwinds_shifted.rename(columns = {'index':'div_winds'})
consecs = divergent_winds.div_winds - divwinds_shifted.div_winds

consec_inds = consecs[consecs != -1].index
divergent_winds_consec = divergent_winds.drop(labels = consec_inds,axis = 0)

In [137]:
## option #2 to get the consecutive indices. 
vals = divergent_winds.index
consecs = []
for i in range(0,len(vals)-1):
    if vals[i]+1==vals[i+1]:
        consecs.append(vals[i])
    else:
        continue